# Computer Vision Analysis
#### By Ronny Toribio, Kadir O. Altunel, Michael Cook-Stahl
#### Based on [Hands on Machine Learning 2nd edition](https://github.com/ageron/handson-ml2/), [FER2013 candidate 1](https://www.kaggle.com/code/ritikjain00/model-training-fer-13) and [FER2013 candidate 2](https://www.kaggle.com/code/gauravsharma99/facial-emotion-recognition/notebook)

### Import modules and declare constants

In [ ]:
%matplotlib inline
import os.path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from scikeras.wrappers import KerasClassifier

### Constants

In [ ]:
BATCH_SIZE = 32 * 1000
IMAGE_ZOOM = 0.3
IMAGE_SHAPE = (48, 48)
INPUT_SHAPE = (48, 48, 1)
TRAIN_DIR = os.path.join("fer2013", "train")
TEST_DIR = os.path.join("fer2013", "test")

### Load Facial Emotion Recognition dataset
#### training, validation, and testing

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=IMAGE_ZOOM, horizontal_flip=True,
                                   validation_split=0.10)
Xy_train = train_datagen.flow_from_directory(TRAIN_DIR, batch_size=BATCH_SIZE, 
                                   target_size=IMAGE_SHAPE, shuffle=True, subset="training",
                                   color_mode="grayscale", class_mode="categorical")

Xy_valid = train_datagen.flow_from_directory(TRAIN_DIR, batch_size=BATCH_SIZE, 
                                   target_size=IMAGE_SHAPE, shuffle=True, subset="validation",
                                   color_mode="grayscale", class_mode="categorical")

test_datagen = ImageDataGenerator(rescale=1./255)
Xy_test = test_datagen.flow_from_directory(TEST_DIR, batch_size=BATCH_SIZE,
                                   target_size=IMAGE_SHAPE, shuffle=True,
                                   color_mode="grayscale", class_mode="categorical")

X_train = Xy_train[0][0]
y_train = Xy_train[0][1]
X_valid = Xy_valid[0][0]
y_valid = Xy_valid[0][1]


### Build and compile CNN model

In [ ]:
def build_model(main_activation, main_initializer, use_conv_block2=False):
    model = Sequential(name="cnn_model")
    # Convolution Block 0
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation=main_activation,
              kernel_initializer= main_initializer, input_shape=INPUT_SHAPE,
              name="conv_block_0_conv_layer_0_input"))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation=main_activation,
              kernel_initializer= main_initializer, name="conv_block_0_conv_layer_1"))
    model.add(BatchNormalization(name="conv_block_0_batchnorm"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid", name="conv_block_0_maxpool"))
    model.add(Dropout(0.25, name="conv_block_0_dropout"))

    # Convolution Block 1
    model.add(Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation=main_activation,
              kernel_initializer= main_initializer, kernel_regularizer=l2(0.01), name="conv_block_1_conv_layer_0"))
    model.add(Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation=main_activation,
              kernel_initializer= main_initializer, kernel_regularizer=l2(0.01), name="conv_block_1_conv_layer_1"))
    model.add(BatchNormalization(name="conv_block_1_batchnorm"))
    model.add(MaxPooling2D(pool_size=(2, 2), padding="valid", name="conv_block_1_maxpool"))
    model.add(Dropout(0.25, name="conv_block_1_dropout"))
        
    # Convolution Block 2
    if use_conv_block2:
        model.add(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation=main_activation,
                  kernel_initializer= main_initializer, kernel_regularizer=l2(0.01),
                  name="conv_block_2_conv_layer_0"))
        model.add(Conv2D(filters=1024, kernel_size=(3, 3), padding="same", activation=main_activation,
                  kernel_initializer= main_initializer, kernel_regularizer=l2(0.01), 
                  name="conv_block_2_conv_layer_1"))
        model.add(BatchNormalization(name="conv_block_2_batchnorm"))
        model.add(MaxPooling2D(pool_size=(2, 2), padding="valid", name="conv_block_2_maxpool"))
        model.add(Dropout(0.25, name="conv_block_2_dropout"))

    # Classification Block
    model.add(Flatten(name="class_block_flatten"))
    model.add(Dense(1024, activation=main_activation, kernel_initializer="he_normal",
              name="class_block_flatten_dense_0"))
    model.add(BatchNormalization(name="class_block_batchnorm"))
    model.add(Dropout(0.5, name="class_block_dropout"))
    model.add(Dense(7, activation="softmax", name="class_block_dense_output"))
    model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.0001, decay=1e-6),
              metrics=["accuracy"], run_eagerly=True)
    return model

### Early stopping callback

In [ ]:
early_stopping_cb = EarlyStopping(min_delta=0.00005, patience=11, verbose=1, restore_best_weights=True)

### Reduce learning rate on plateau callback

In [ ]:
reduce_lr_cb = ReduceLROnPlateau(factor=0.5, patience=7, min_l=1e-7, verbose=1)

### Grid Search Hyperparameters for our model

In [ ]:
param_grid = [
    {"main_activation": ["relu", "elu"], "main_initializer":["he_normal"], "use_conv_block2": [True, False]},
    {"main_activation": ["selu"], "main_initializer":["lecun_normal"], "use_conv_block2": [True, False]}
]

keras_clf = KerasClassifier(
    model=build_model,
    main_activation="relu",
    main_initializer="he_normal",
    use_conv_block2=False
)

grid_search = GridSearchCV(keras_clf, param_grid, scoring='neg_mean_squared_error', return_train_score=True)                      
grid_search.fit(X_train,
                y_train,
                epochs=60,
                validation_data=(X_valid, y_valid),
                steps_per_epoch=Xy_train.n // BATCH_SIZE,
                validation_steps=Xy_valid.n // BATCH_SIZE,
                callbacks=[early_stopping_cb, reduce_lr_cb])
model = grid_search.best_estimator_.model
history = grid_search.best_estimator_.history

### Grid Search Results

In [ ]:
print("Best Hyperparameters: {}".format(grid_search.best_params_))
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

### Save and plot training history

In [ ]:
training = pd.DataFrame(history.history, history.epoch)
training.to_csv("cnn_model_training.txt")
ax.grid(True)
training.plot()
plt.savefig("cnn_model_training.png")
plt.show()

### Model properties

In [ ]:
model.layers

In [ ]:
model.summary()

### Save model architecture as JSON

In [ ]:
model_json = model.to_json()
with open("cnn_model.json", "w") as f:
    f.write(model_json)

### Save model weights as HDF5

In [ ]:
model.save_weights("cnn_model_weights.h5")

### Evaluate model

In [ ]:
print("Evaluate train set")
model.evaluate(Xy_train)
print("Evaluate validation set")
model.evaluate(Xy_valid)
print("Evaluate test set")
model.evaluate(Xy_test)